## Import Required Libraries

In [16]:
import pandas as pd
import json
import os
from pathlib import Path
from typing import Dict, List, Any
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## Configuration

In [17]:
# Path to audit records directory
AUDIT_RECORDS_DIR = Path(r"d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Streamlit app\audit_records")
OUTPUT_DIR = Path(r"d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data")

# Create output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Target workflows to filter
TARGET_WORKFLOWS = ['new_prompt_analysis', 'dataset_prompt_analysis']

print(f"Audit records directory: {AUDIT_RECORDS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Target workflows: {TARGET_WORKFLOWS}")

Audit records directory: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Streamlit app\audit_records
Output directory: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data
Target workflows: ['new_prompt_analysis', 'dataset_prompt_analysis']


## Load JSON Files

Recursively load all JSON files from the audit_records directory.

In [18]:
def load_audit_records(audit_dir: Path) -> List[Dict[str, Any]]:
    """
    Load all JSON files from audit_records directory recursively.
    Only loads files that start with 'audit'.
    
    Args:
        audit_dir: Path to the audit records directory
        
    Returns:
        List of parsed JSON records
    """
    records = []
    json_files = list(audit_dir.rglob('audit*.json'))
    
    print(f"Found {len(json_files)} JSON files starting with 'audit'")
    
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Add file path for reference
                data['_source_file'] = str(json_file)
                records.append(data)
        except json.JSONDecodeError as e:
            print(f"Error parsing {json_file}: {e}")
        except Exception as e:
            print(f"Error loading {json_file}: {e}")
    
    print(f"Successfully loaded {len(records)} records")
    return records

# Load all records
all_records = load_audit_records(AUDIT_RECORDS_DIR)
print(f"\nTotal records loaded: {len(all_records)}")

Found 84 JSON files starting with 'audit'
Successfully loaded 84 records

Total records loaded: 84


## Filter Records

Keep only records where workflow is either `new_prompt_analysis` or `dataset_prompt_analysis`.

In [19]:
def filter_analysis_workflows(records: List[Dict], target_workflows: List[str]) -> List[Dict]:
    """
    Filter records to keep only analysis workflows.
    
    Args:
        records: List of all audit records
        target_workflows: List of workflow names to keep
        
    Returns:
        Filtered list of records
    """
    filtered = []
    
    for record in records:
        workflow = record.get('workflow', '')
        if workflow in target_workflows:
            filtered.append(record)
    
    return filtered

# Filter records
analysis_records = filter_analysis_workflows(all_records, TARGET_WORKFLOWS)

print(f"Filtered records: {len(analysis_records)}")
print(f"\nWorkflow distribution:")
workflow_counts = {}
for record in analysis_records:
    workflow = record.get('workflow', 'unknown')
    workflow_counts[workflow] = workflow_counts.get(workflow, 0) + 1

for workflow, count in workflow_counts.items():
    print(f"  {workflow}: {count}")

Filtered records: 84

Workflow distribution:
  dataset_prompt_analysis: 42
  new_prompt_analysis: 42


## Extract Workflow Data

Group by workflow_id and extract all required fields into a structured format.

In [20]:
def safe_get(dictionary: Dict, *keys, default=None):
    """
    Safely get nested dictionary values.
    
    Args:
        dictionary: Input dictionary
        *keys: Keys to traverse
        default: Default value if key doesn't exist
        
    Returns:
        Value at the nested key, or default
    """
    current = dictionary
    for key in keys:
        if isinstance(current, dict):
            current = current.get(key, default)
        else:
            return default
    return current if current is not None else default


def extract_cwe_list(vulnerabilities: list) -> list:
    """
    Extract unique CWE IDs from vulnerability list.
    
    Args:
        vulnerabilities: List of vulnerability dictionaries
        
    Returns:
        Sorted list of unique CWE IDs
    """
    if not vulnerabilities:
        return []
    
    cwe_ids = set()
    for vuln in vulnerabilities:
        cwe_id = vuln.get('cwe_id', vuln.get('test_id', ''))
        if cwe_id:
            # Normalize CWE ID (remove 'CWE-' prefix if present)
            cwe_id = str(cwe_id).replace('CWE-', '').strip()
            cwe_ids.add(cwe_id)
    
    return sorted(list(cwe_ids))


def count_fix_providers_by_cwe(record: Dict) -> Dict[str, Any]:
    """
    Count fixes by provider (LLM vs custom rule) and track which unique CWEs were fixed by each.
    
    Args:
        record: Audit record
        
    Returns:
        Dictionary with counts for each fix provider and CWE lists
    """
    llm_cwes = set()
    rule_cwes = set()
    unknown_cwes = set()
    
    # Check initial patch result
    initial_changes = safe_get(record, 'initial_patch_result', 'changes', default=[])
    for change in initial_changes:
        patch_method = change.get('patch_method', '').lower()
        cwe_id = change.get('cwe_id', '')
        if cwe_id:
            cwe_id = str(cwe_id).replace('CWE-', '').strip()
            if 'llm' in patch_method:
                llm_cwes.add(cwe_id)
            elif 'rule' in patch_method:
                rule_cwes.add(cwe_id)
            else:
                unknown_cwes.add(cwe_id)
    
    # Check patch iterations
    patch_iterations = safe_get(record, 'patch_iterations', default=[])
    for iteration in patch_iterations:
        changes = iteration.get('changes', [])
        for change in changes:
            patch_method = change.get('patch_method', '').lower()
            cwe_id = change.get('cwe_id', '')
            if cwe_id:
                cwe_id = str(cwe_id).replace('CWE-', '').strip()
                if 'llm' in patch_method:
                    llm_cwes.add(cwe_id)
                elif 'rule' in patch_method:
                    rule_cwes.add(cwe_id)
                else:
                    unknown_cwes.add(cwe_id)
    
    return {
        'llm': len(llm_cwes),
        'rule_based': len(rule_cwes),
        'unknown': len(unknown_cwes),
        'llm_cwes': sorted(list(llm_cwes)),
        'rule_cwes': sorted(list(rule_cwes)),
        'unknown_cwes': sorted(list(unknown_cwes))
    }


def extract_workflow_data(record: Dict) -> Dict[str, Any]:
    """
    Extract all required fields from a single workflow record.
    
    Args:
        record: Audit record (can be flat or nested format)
        
    Returns:
        Dictionary with extracted workflow data
    """
    # Check if this is already a flattened audit record
    # (has fields like 'initial_detection_bandit_count' at top level)
    is_flat = 'initial_detection_bandit_count' in record
    
    if is_flat:
        # Data is already in the correct format, just return it
        workflow = record.get('workflow', '')
        return {
            'workflow_id': record.get('workflow_id', ''),
            'workflow': workflow,
            'timestamp': record.get('timestamp', ''),
            'file': record.get('file', ''),
            'source_file': record.get('source_file', ''),
            'prompt_type': 'Manual' if workflow == 'new_prompt_analysis' else 'SecurityEval',
            'prompt': record.get('original_content', ''),
            'llm_response': record.get('llm_response', ''),
            'vulnerabilities_found': record.get('vulnerabilities_found', 0),
            'total_vulnerabilities_identified': record.get('total_vulnerabilities_identified', 0),
            'total_vulnerabilities_fixed': record.get('total_vulnerabilities_fixed', 0),
            'total_vulnerabilities_remaining': record.get('total_vulnerabilities_remaining', 0),
            'initial_detection_bandit_count': record.get('initial_detection_bandit_count', 0),
            'initial_detection_bandit_cwes': record.get('initial_detection_bandit_cwes', []),
            'initial_detection_semgrep_count': record.get('initial_detection_semgrep_count', 0),
            'initial_detection_semgrep_cwes': record.get('initial_detection_semgrep_cwes', []),
            'initial_detection_ast_count': record.get('initial_detection_ast_count', 0),
            'initial_detection_ast_cwes': record.get('initial_detection_ast_cwes', []),
            'iteration_detection_bandit_count': record.get('iteration_detection_bandit_count', 0),
            'iteration_detection_bandit_cwes': record.get('iteration_detection_bandit_cwes', []),
            'iteration_detection_semgrep_count': record.get('iteration_detection_semgrep_count', 0),
            'iteration_detection_semgrep_cwes': record.get('iteration_detection_semgrep_cwes', []),
            'iteration_detection_ast_count': record.get('iteration_detection_ast_count', 0),
            'iteration_detection_ast_cwes': record.get('iteration_detection_ast_cwes', []),
            'iterations_count': record.get('iterations_count', 0),
            'fixed_cwes': record.get('fixed_cwes', []),
            'remaining_cwes': record.get('remaining_cwes', []),
            'fix_provider_llm': record.get('fix_provider_llm', 0),
            'fix_provider_rule_based': record.get('fix_provider_rule_based', 0),
            'fix_provider_unknown': record.get('fix_provider_unknown', 0),
            'fix_provider_llm_cwes': record.get('fix_provider_llm_cwes', []),
            'fix_provider_rule_cwes': record.get('fix_provider_rule_cwes', []),
            'fix_provider_unknown_cwes': record.get('fix_provider_unknown_cwes', []),
        }
    
    # Otherwise, handle nested format (original workflow records)
    # Basic information
    workflow = record.get('workflow', '')
    data = {
        'workflow_id': record.get('workflow_id', ''),
        'workflow': workflow,
        'timestamp': record.get('timestamp', ''),
        'file': record.get('file', ''),
        'source_file': record.get('_source_file', ''),
        'prompt_type': 'Manual' if workflow == 'new_prompt_analysis' else 'SecurityEval',
    }
    
    # Prompt and response
    data['prompt'] = record.get('content', record.get('original_content', ''))
    data['llm_response'] = record.get('response', '')
    
    # Vulnerability counts (these include duplicates)
    data['vulnerabilities_found'] = record.get('vulnerabilities_found', 0)
    data['total_vulnerabilities_identified'] = record.get('total_vulnerabilities_identified', 0)
    data['total_vulnerabilities_fixed'] = record.get('total_vulnerabilities_fixed', 0)
    data['total_vulnerabilities_remaining'] = record.get('total_vulnerabilities_remaining', 0)
    
    # Initial detection by tool
    initial_run = record.get('initial_run_by_tool', {})
    
    # Bandit
    bandit_vulns = safe_get(initial_run, 'bandit', 'identified_vulnerabilities', default=[])
    data['initial_detection_bandit_count'] = safe_get(initial_run, 'bandit', 'count', default=0)
    data['initial_detection_bandit_cwes'] = extract_cwe_list(bandit_vulns)
    
    # Semgrep
    semgrep_vulns = safe_get(initial_run, 'semgrep', 'identified_vulnerabilities', default=[])
    data['initial_detection_semgrep_count'] = safe_get(initial_run, 'semgrep', 'count', default=0)
    data['initial_detection_semgrep_cwes'] = extract_cwe_list(semgrep_vulns)
    
    # AST (custom_detector)
    ast_vulns = safe_get(initial_run, 'custom_detector', 'identified_vulnerabilities', default=[])
    data['initial_detection_ast_count'] = safe_get(initial_run, 'custom_detector', 'count', default=0)
    data['initial_detection_ast_cwes'] = extract_cwe_list(ast_vulns)
    
    # Iteration detection by tool
    iterations_by_tool = record.get('iterations_by_tool', {})
    
    # Bandit iterations
    bandit_iter_vulns = safe_get(iterations_by_tool, 'bandit', 'identified_vulnerabilities', default=[])
    data['iteration_detection_bandit_count'] = safe_get(iterations_by_tool, 'bandit', 'total_across_all_iterations', default=0)
    data['iteration_detection_bandit_cwes'] = extract_cwe_list(bandit_iter_vulns)
    
    # Semgrep iterations
    semgrep_iter_vulns = safe_get(iterations_by_tool, 'semgrep', 'identified_vulnerabilities', default=[])
    data['iteration_detection_semgrep_count'] = safe_get(iterations_by_tool, 'semgrep', 'total_across_all_iterations', default=0)
    data['iteration_detection_semgrep_cwes'] = extract_cwe_list(semgrep_iter_vulns)
    
    # AST iterations
    ast_iter_vulns = safe_get(iterations_by_tool, 'custom_detector', 'identified_vulnerabilities', default=[])
    data['iteration_detection_ast_count'] = safe_get(iterations_by_tool, 'custom_detector', 'total_across_all_iterations', default=0)
    data['iteration_detection_ast_cwes'] = extract_cwe_list(ast_iter_vulns)
    
    # Iterations count
    data['iterations_count'] = len(record.get('patch_iterations', []))
    
    # Fixed and remaining CWEs (these are unique)
    data['fixed_cwes'] = record.get('fixed_cwe_ids', [])
    data['remaining_cwes'] = record.get('non_fixed_cwe_ids', [])
    
    # Fix provider counts by unique CWE
    fix_providers = count_fix_providers_by_cwe(record)
    data['fix_provider_llm'] = fix_providers['llm']
    data['fix_provider_rule_based'] = fix_providers['rule_based']
    data['fix_provider_unknown'] = fix_providers['unknown']
    data['fix_provider_llm_cwes'] = fix_providers['llm_cwes']
    data['fix_provider_rule_cwes'] = fix_providers['rule_cwes']
    data['fix_provider_unknown_cwes'] = fix_providers['unknown_cwes']
    
    return data

print("Data extraction functions defined successfully!")

Data extraction functions defined successfully!


## Build DataFrame

Process all filtered records and create a DataFrame with one row per workflow.

In [21]:
# Extract data from all analysis records
workflow_data = []

for record in analysis_records:
    try:
        data = extract_workflow_data(record)
        workflow_data.append(data)
    except Exception as e:
        print(f"Error processing record {record.get('workflow_id', 'unknown')}: {e}")

# Create DataFrame
df = pd.DataFrame(workflow_data)

print(f"DataFrame created with {len(df)} rows and {len(df.columns)} columns")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn names:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

DataFrame created with 84 rows and 33 columns

DataFrame shape: (84, 33)

Column names:
  1. workflow_id
  2. workflow
  3. timestamp
  4. file
  5. source_file
  6. prompt_type
  7. prompt
  8. llm_response
  9. vulnerabilities_found
  10. total_vulnerabilities_identified
  11. total_vulnerabilities_fixed
  12. total_vulnerabilities_remaining
  13. initial_detection_bandit_count
  14. initial_detection_bandit_cwes
  15. initial_detection_semgrep_count
  16. initial_detection_semgrep_cwes
  17. initial_detection_ast_count
  18. initial_detection_ast_cwes
  19. iteration_detection_bandit_count
  20. iteration_detection_bandit_cwes
  21. iteration_detection_semgrep_count
  22. iteration_detection_semgrep_cwes
  23. iteration_detection_ast_count
  24. iteration_detection_ast_cwes
  25. iterations_count
  26. fixed_cwes
  27. remaining_cwes
  28. fix_provider_llm
  29. fix_provider_rule_based
  30. fix_provider_unknown
  31. fix_provider_llm_cwes
  32. fix_provider_rule_cwes
  33. fix_prov

In [22]:
def clean_and_standardize_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and standardize the DataFrame.
    
    Args:
        df: Input DataFrame
        
    Returns:
        Cleaned DataFrame
    """
    df_clean = df.copy()
    
    # Ensure list columns are actual lists (not strings or NaN)
    list_columns = [
        'initial_detection_bandit_cwes',
        'initial_detection_semgrep_cwes',
        'initial_detection_ast_cwes',
        'iteration_detection_bandit_cwes',
        'iteration_detection_semgrep_cwes',
        'iteration_detection_ast_cwes',
        'fixed_cwes',
        'remaining_cwes',
        'fix_provider_llm_cwes',
        'fix_provider_rule_cwes',
        'fix_provider_unknown_cwes'
    ]
    
    for col in list_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].apply(
                lambda x: x if isinstance(x, list) else ([] if pd.isna(x) else [])
            )
    
    # Ensure count columns are integers
    count_columns = [
        'vulnerabilities_found',
        'total_vulnerabilities_identified',
        'total_vulnerabilities_fixed',
        'total_vulnerabilities_remaining',
        'initial_detection_bandit_count',
        'initial_detection_semgrep_count',
        'initial_detection_ast_count',
        'iteration_detection_bandit_count',
        'iteration_detection_semgrep_count',
        'iteration_detection_ast_count',
        'iterations_count',
        'fix_provider_llm',
        'fix_provider_rule_based',
        'fix_provider_unknown'
    ]
    
    for col in count_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0).astype(int)
    
    # Handle missing string values
    string_columns = ['workflow_id', 'workflow', 'timestamp', 'file', 'source_file', 'prompt', 'llm_response', 'prompt_type']
    for col in string_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].fillna('')
    
    # Convert timestamp to datetime
    if 'timestamp' in df_clean.columns:
        df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'], errors='coerce')
    
    # Add computed columns for unique CWE counts
    if 'fixed_cwes' in df_clean.columns:
        df_clean['unique_cwes_fixed'] = df_clean['fixed_cwes'].apply(len)
    
    if 'remaining_cwes' in df_clean.columns:
        df_clean['unique_cwes_remaining'] = df_clean['remaining_cwes'].apply(len)
    
    # Calculate total unique CWEs identified (fixed + remaining)
    if 'fixed_cwes' in df_clean.columns and 'remaining_cwes' in df_clean.columns:
        df_clean['unique_cwes_identified'] = df_clean.apply(
            lambda row: len(set(row['fixed_cwes']) | set(row['remaining_cwes'])),
            axis=1
        )
    
    return df_clean

# Apply cleaning
df = clean_and_standardize_dataframe(df)

print("DataFrame cleaned and standardized successfully!")
print(f"\nUpdated shape: {df.shape}")
print(f"\nNew computed columns added:")
print("  - unique_cwes_identified: Count of unique CWEs found")
print("  - unique_cwes_fixed: Count of unique CWEs fixed")
print("  - unique_cwes_remaining: Count of unique CWEs remaining")

DataFrame cleaned and standardized successfully!

Updated shape: (84, 36)

New computed columns added:
  - unique_cwes_identified: Count of unique CWEs found
  - unique_cwes_fixed: Count of unique CWEs fixed
  - unique_cwes_remaining: Count of unique CWEs remaining


## Data Cleaning and Standardization

Clean and standardize the DataFrame to ensure consistent data types and handle missing values.

In [23]:
# Export complete dataset
evaluation_base_path = OUTPUT_DIR / 'evaluation_base.csv'
df.to_csv(evaluation_base_path, index=False)
print(f"✓ Complete dataset saved to: {evaluation_base_path}")
print(f"  Total workflows: {len(df)}")

# Export manual prompts only
df_manual = df[df['prompt_type'] == 'Manual']
manual_base_path = OUTPUT_DIR / 'manual_base.csv'
df_manual.to_csv(manual_base_path, index=False)
print(f"\n✓ Manual dataset saved to: {manual_base_path}")
print(f"  Manual workflows: {len(df_manual)}")

# Export SecurityEval prompts only
df_securityeval = df[df['prompt_type'] == 'SecurityEval']
securityeval_base_path = OUTPUT_DIR / 'securityeval_base.csv'
df_securityeval.to_csv(securityeval_base_path, index=False)
print(f"\n✓ SecurityEval dataset saved to: {securityeval_base_path}")
print(f"  SecurityEval workflows: {len(df_securityeval)}")

print("\n" + "=" * 80)
print("EXPORT SUMMARY")
print("=" * 80)
print(f"Total workflows exported: {len(df)}")
print(f"  Manual: {len(df_manual)}")
print(f"  SecurityEval: {len(df_securityeval)}")
print(f"\nAll datasets exported successfully!")
print("=" * 80)

✓ Complete dataset saved to: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\evaluation_base.csv
  Total workflows: 84

✓ Manual dataset saved to: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\manual_base.csv
  Manual workflows: 42

✓ SecurityEval dataset saved to: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\securityeval_base.csv
  SecurityEval workflows: 42

EXPORT SUMMARY
Total workflows exported: 84
  Manual: 42
  SecurityEval: 42

All datasets exported successfully!


## Export Datasets

Export the cleaned DataFrame to multiple CSV files:
- `evaluation_base.csv`: Complete dataset
- `manual_base.csv`: Manual prompts only
- `securityeval_base.csv`: SecurityEval prompts only

In [24]:
print("=" * 80)
print("SUMMARY BY PROMPT TYPE")
print("=" * 80)

print(f"\nTotal workflows: {len(df)}")

print("\nWorkflows by prompt type:")
prompt_type_counts = df['prompt_type'].value_counts()
for prompt_type, count in prompt_type_counts.items():
    print(f"  {prompt_type}: {count}")

print("\nDetailed statistics by prompt type:")
print("-" * 80)

for prompt_type in df['prompt_type'].unique():
    df_subset = df[df['prompt_type'] == prompt_type]
    print(f"\n{prompt_type}:")
    print(f"  Workflows: {len(df_subset)}")
    print(f"  Total vulnerabilities identified: {df_subset['total_vulnerabilities_identified'].sum()}")
    print(f"  Total vulnerabilities fixed: {df_subset['total_vulnerabilities_fixed'].sum()}")
    print(f"  Total vulnerabilities remaining: {df_subset['total_vulnerabilities_remaining'].sum()}")
    print(f"  Average iterations: {df_subset['iterations_count'].mean():.2f}")
    print(f"  LLM fixes: {df_subset['fix_provider_llm'].sum()}")
    print(f"  Rule-based fixes: {df_subset['fix_provider_rule_based'].sum()}")

print("\n" + "=" * 80)

SUMMARY BY PROMPT TYPE

Total workflows: 84

Workflows by prompt type:
  SecurityEval: 42
  Manual: 42

Detailed statistics by prompt type:
--------------------------------------------------------------------------------

SecurityEval:
  Workflows: 42
  Total vulnerabilities identified: 95
  Total vulnerabilities fixed: 91
  Total vulnerabilities remaining: 4
  Average iterations: 1.07
  LLM fixes: 30
  Rule-based fixes: 45

Manual:
  Workflows: 42
  Total vulnerabilities identified: 120
  Total vulnerabilities fixed: 111
  Total vulnerabilities remaining: 9
  Average iterations: 0.98
  LLM fixes: 17
  Rule-based fixes: 67



## Summary by Prompt Type

Display summary statistics grouped by prompt type.

## Data Overview

Display basic information about the prepared dataset.

## Key Columns Reference

### Vulnerability Counts - Two Perspectives:

**Total Counts** (may include duplicates):
- `total_vulnerabilities_identified`: All vulnerability occurrences found
- `total_vulnerabilities_fixed`: All vulnerability occurrences fixed
- `total_vulnerabilities_remaining`: All vulnerability occurrences not fixed

**Unique CWE Counts** (no duplicates):
- `unique_cwes_identified`: Distinct CWE types found
- `unique_cwes_fixed`: Distinct CWE types fixed
- `unique_cwes_remaining`: Distinct CWE types not fixed

### Fix Provider Columns:
- `fix_provider_llm`: Count of unique CWEs fixed by LLM
- `fix_provider_rule_based`: Count of unique CWEs fixed by rules
- `fix_provider_unknown`: Count of unique CWEs fixed by unknown method
- `fix_provider_llm_cwes`: List of specific CWEs fixed by LLM
- `fix_provider_rule_cwes`: List of specific CWEs fixed by rules
- `fix_provider_unknown_cwes`: List of specific CWEs fixed by unknown method

### Important Note:
**Fix provider counts align with unique CWE counts**, not total counts. Use `unique_cwes_*` columns for analysis involving fix providers.

In [25]:
# Display full table with ALL columns visible
print(f"Dataset contains {len(df)} workflows with {len(df.columns)} columns\n")

# Configure pandas display options to show ALL data
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.width', 1000)        # Wide display
pd.set_option('display.max_colwidth', 100)  # Limit cell width for readability

# Option 1: Display full dataframe (scrollable)
print("Full Dataset (scroll right to see all columns):")
print("=" * 120)
display(df)

# Option 2: Display as transposed view (easier to see all columns)
print("\n\nTransposed View (first 3 records - easier to see all columns):")
print("=" * 120)
display(df.head(3).T)

print("\n\nMissing values summary:")
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("No missing values found")

Dataset contains 84 workflows with 36 columns

Full Dataset (scroll right to see all columns):


,workflow_id,workflow,timestamp,file,source_file,prompt_type,prompt,llm_response,vulnerabilities_found,total_vulnerabilities_identified,total_vulnerabilities_fixed,total_vulnerabilities_remaining,initial_detection_bandit_count,initial_detection_bandit_cwes,initial_detection_semgrep_count,initial_detection_semgrep_cwes,initial_detection_ast_count,initial_detection_ast_cwes,iteration_detection_bandit_count,iteration_detection_bandit_cwes,iteration_detection_semgrep_count,iteration_detection_semgrep_cwes,iteration_detection_ast_count,iteration_detection_ast_cwes,iterations_count,fixed_cwes,remaining_cwes,fix_provider_llm,fix_provider_rule_based,fix_provider_unknown,fix_provider_llm_cwes,fix_provider_rule_cwes,fix_provider_unknown_cwes,unique_cwes_fixed,unique_cwes_remaining,unique_cwes_identified
0,20251219_151500,dataset_prompt_analysis,2025-12-19 14:15:29.482228+00:00,CWE-020/author_1.py,CWE-020/author_1.py,SecurityEval,import yaml\n\n\ndef yaml_load(filename):\n '''\n Use the filename variable to open a file...,"def yaml_load(filename):\n with open(filename, 'r') as f:\n return yaml.safe_load(f)\n",2,2,2,0,0,[],0,[],2,"[022, 703]",0,[],0,[],0,[],0,"[022, 703]",[],0,2,0,[],[],[],2,0,2
1,20251219_152650,dataset_prompt_analysis,2025-12-19 14:27:08.255669+00:00,CWE-020/author_2.py,CWE-020/author_2.py,SecurityEval,import xml.etree.ElementTree as ET\n\n\ndef parse_xml(xml_string):\n '''\n Parse xml_strin...,def parse_xml(xml_string):\n return ET.fromstring(xml_string),0,0,0,0,0,[],0,[],0,[],0,[],0,[],0,[],0,[],[],0,0,0,[],[],[],0,0,0
2,20251219_154533,dataset_prompt_analysis,2025-12-19 14:46:30.850674+00:00,CWE-022/author_1.py,CWE-022/author_1.py,SecurityEval,"import os\nfrom flask import Flask, request\n\napp = Flask(__name__)\n\n@app.route('/deleteImage...","@app.route('/deleteImage', methods=['POST'])\ndef deleteImage():\n filename = request.form.ge...",3,3,3,0,0,[],0,[],1,[352],0,[],0,[],2,[352],2,[352],[],3,0,0,[],[],[],1,0,1
3,20251219_155128,dataset_prompt_analysis,2025-12-19 14:51:43.477065+00:00,CWE-022/author_2.py,CWE-022/author_2.py,SecurityEval,import json\nimport urllib.request as request\ndef readJSON(url):\n '''\n Read a json file...,def readJSON(url):\n with request.urlopen(url) as response:\n return json.load(response),0,0,0,0,0,[],0,[],0,[],0,[],0,[],0,[],0,[],[],0,0,0,[],[],[],0,0,0
4,20251219_155343,dataset_prompt_analysis,2025-12-19 14:53:57.481675+00:00,CWE-078/author_1.py,CWE-078/author_1.py,SecurityEval,"import subprocess\n\ndef executeCommand():\n '''\n Read a command from a user input, valid...",import shlex\nimport shutil\n\ndef executeCommand():\n '''\n Read a command from a user in...,0,0,0,0,0,[],0,[],0,[],0,[],0,[],0,[],0,[],[],0,0,0,[],[],[],0,0,0
5,20251219_164453,dataset_prompt_analysis,2025-12-19 15:45:48.643410+00:00,CWE-080/author_1.py,CWE-080/author_1.py,SecurityEval,"from flask import Flask, request, make_response\napp = Flask(__name__)\n\n@app.route(""/renderHtm...","@app.route(""/renderHtml"")\ndef renderHTML():\n name = request.args.get(""name"", ""World"")\n ...",6,6,6,0,0,[],1,[079],1,[080],0,[],3,"[079, 096]",1,[080],2,"[079, 080, 096]",[],3,1,0,[],[],[],3,0,3
6,20251219_164921,dataset_prompt_analysis,2025-12-19 15:49:36.669443+00:00,CWE-089/author_1.py,CWE-089/author_1.py,SecurityEval,import mysql.connector\n\n\ndef get_connection():\n '''\n Return a connection to the datab...,"def remove_user(username):\n '''\n Remove the user with the ""username"" specified from the ...",0,0,0,0,0,[],0,[],0,[],0,[],0,[],0,[],0,[],[],0,0,0,[],[],[],0,0,0
7,20251219_165024,dataset_prompt_analysis,2025-12-19 15:50:48.920162+00:00,CWE-094/author_1.py,CWE-094/author_1.py,SecurityEval,"from flask import Flask, request\napp = Flask(__name__)\n\n@app.route(""/evalcode"")\ndef eval_cod...","from flask import Flask, request\napp = Flask(__name__)\n\n@app.route(""/evalcode"")\ndef eval_cod...",5,5,5,0,1,[078],1,[095],3,"[094, 095]",0,[],0,[],0,[],0,"[078, 094, 095]",[],0,3,0,[],[],[],3,0,3
8,20251219_165140,datas



Transposed View (first 3 records - easier to see all columns):


,0,1,2
workflow_id,20251219_151500,20251219_152650,20251219_154533
workflow,dataset_prompt_analysis,dataset_prompt_analysis,dataset_prompt_analysis
timestamp,2025-12-19 14:15:29.482228+00:00,2025-12-19 14:27:08.255669+00:00,2025-12-19 14:46:30.850674+00:00
file,CWE-020/author_1.py,CWE-020/author_2.py,CWE-022/author_1.py
source_file,CWE-020/author_1.py,CWE-020/author_2.py,CWE-022/author_1.py
prompt_type,SecurityEval,SecurityEval,SecurityEval
prompt,import yaml\n\n\ndef yaml_load(filename):\n '''\n Use the filename variable to open a file...,import xml.etree.ElementTree as ET\n\n\ndef parse_xml(xml_string):\n '''\n Parse xml_strin...,"import os\nfrom flask import Flask, request\n\napp = Flask(__name__)\n\n@app.route('/deleteImage..."
llm_response,"def yaml_load(filename):\n with open(filename, 'r') as f:\n return yaml.safe_load(f)\n",def parse_xml(xml_string):\n return ET.fromstring(xml_string),"@app.route('/deleteImage', methods=['POST'])\ndef deleteImage():\n filename = request.form.ge..."
vulnerabilities_found,2,0,3
total_vulnerabilities_identified,2,0,3




Missing values summary:
No missing values found


## Summary Statistics

In [26]:
print("=" * 80)
print("DATASET SUMMARY")
print("=" * 80)

print(f"\nTotal workflows: {len(df)}")

print("\nWorkflow type distribution:")
print(df['workflow'].value_counts())

print("\n" + "-" * 80)
print("VULNERABILITY DETECTION SUMMARY")
print("-" * 80)

print("\nTotal Occurrences (includes duplicates):")
print(f"  Vulnerabilities identified: {df['total_vulnerabilities_identified'].sum()}")
print(f"  Vulnerabilities fixed: {df['total_vulnerabilities_fixed'].sum()}")
print(f"  Vulnerabilities remaining: {df['total_vulnerabilities_remaining'].sum()}")

if 'unique_cwes_identified' in df.columns:
    print("\nUnique CWE Types (no duplicates):")
    print(f"  Unique CWEs identified: {df['unique_cwes_identified'].sum()}")
    print(f"  Unique CWEs fixed: {df['unique_cwes_fixed'].sum()}")
    print(f"  Unique CWEs remaining: {df['unique_cwes_remaining'].sum()}")
    
    # Calculate fix rate
    total_unique_identified = df['unique_cwes_identified'].sum()
    total_unique_fixed = df['unique_cwes_fixed'].sum()
    if total_unique_identified > 0:
        fix_rate = (total_unique_fixed / total_unique_identified) * 100
        print(f"\n  Fix success rate: {fix_rate:.1f}% ({total_unique_fixed}/{total_unique_identified})")

print("\n" + "-" * 80)
print("DETECTION TOOL STATISTICS (Initial Run)")
print("-" * 80)
print(f"  Bandit detections: {df['initial_detection_bandit_count'].sum()}")
print(f"  Semgrep detections: {df['initial_detection_semgrep_count'].sum()}")
print(f"  AST detections: {df['initial_detection_ast_count'].sum()}")

print("\n" + "-" * 80)
print("ITERATION STATISTICS")
print("-" * 80)
print(f"  Total iterations: {df['iterations_count'].sum()}")
print(f"  Average iterations per workflow: {df['iterations_count'].mean():.2f}")
print(f"  Max iterations: {df['iterations_count'].max()}")

print("\n" + "-" * 80)
print("FIX PROVIDER DISTRIBUTION (Unique CWEs)")
print("-" * 80)
llm_fixes = df['fix_provider_llm'].sum()
rule_fixes = df['fix_provider_rule_based'].sum()
unknown_fixes = df['fix_provider_unknown'].sum()
total_fixes = llm_fixes + rule_fixes + unknown_fixes

print(f"  LLM fixes: {llm_fixes}")
print(f"  Rule-based fixes: {rule_fixes}")
print(f"  Unknown provider: {unknown_fixes}")
print(f"  Total: {total_fixes}")

if total_fixes > 0:
    print(f"\n  Fix provider percentages:")
    print(f"    LLM: {(llm_fixes/total_fixes)*100:.1f}%")
    print(f"    Rule-based: {(rule_fixes/total_fixes)*100:.1f}%")
    print(f"    Unknown: {(unknown_fixes/total_fixes)*100:.1f}%")

print("\n" + "=" * 80)

DATASET SUMMARY

Total workflows: 84

Workflow type distribution:
workflow
dataset_prompt_analysis    42
new_prompt_analysis        42
Name: count, dtype: int64

--------------------------------------------------------------------------------
VULNERABILITY DETECTION SUMMARY
--------------------------------------------------------------------------------

Total Occurrences (includes duplicates):
  Vulnerabilities identified: 215
  Vulnerabilities fixed: 202
  Vulnerabilities remaining: 13

Unique CWE Types (no duplicates):
  Unique CWEs identified: 59
  Unique CWEs fixed: 48
  Unique CWEs remaining: 11

  Fix success rate: 81.4% (48/59)

--------------------------------------------------------------------------------
DETECTION TOOL STATISTICS (Initial Run)
--------------------------------------------------------------------------------
  Bandit detections: 26
  Semgrep detections: 15
  AST detections: 29

--------------------------------------------------------------------------------
I

## Save Clean Dataset

In [27]:
# Save to CSV
csv_path = OUTPUT_DIR / 'workflow_analysis_dataset.csv'
df.to_csv(csv_path, index=False)
print(f"Dataset saved to: {csv_path}")

# Save to Parquet (more efficient for larger datasets)
parquet_path = OUTPUT_DIR / 'workflow_analysis_dataset.parquet'
df.to_parquet(parquet_path, index=False)
print(f"Dataset saved to: {parquet_path}")

# Save to Excel for easy viewing (optional, requires openpyxl)
try:
    excel_path = OUTPUT_DIR / 'workflow_analysis_dataset.xlsx'
    df.to_excel(excel_path, index=False, engine='openpyxl')
    print(f"Dataset saved to: {excel_path}")
except ImportError:
    print("Note: Excel export skipped (openpyxl not installed)")

print(f"\nAll files saved successfully!")
print(f"Dataset contains {len(df)} workflows with {len(df.columns)} features.")

Dataset saved to: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\workflow_analysis_dataset.csv
Dataset saved to: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\workflow_analysis_dataset.parquet
Note: Excel export skipped (openpyxl not installed)

All files saved successfully!
Dataset contains 84 workflows with 36 features.


## Sample Records Inspection

View a few complete records to verify data quality.

In [28]:
# Display full information for first record
if len(df) > 0:
    print("Sample workflow record (first entry):")
    print("=" * 80)
    sample = df.iloc[0]
    for col in df.columns:
        value = sample[col]
        # Truncate long strings for readability
        if isinstance(value, str) and len(value) > 100:
            value = value[:100] + "..."
        print(f"{col}: {value}")
else:
    print("No records found in dataset.")

Sample workflow record (first entry):
workflow_id: 20251219_151500
workflow: dataset_prompt_analysis
timestamp: 2025-12-19 14:15:29.482228+00:00
file: CWE-020/author_1.py
source_file: CWE-020/author_1.py
prompt_type: SecurityEval
prompt: import yaml


def yaml_load(filename):
    '''
    Use the filename variable to open a file, 
    lo...
llm_response: def yaml_load(filename):
    with open(filename, 'r') as f:
        return yaml.safe_load(f)

vulnerabilities_found: 2
total_vulnerabilities_identified: 2
total_vulnerabilities_fixed: 2
total_vulnerabilities_remaining: 0
initial_detection_bandit_count: 0
initial_detection_bandit_cwes: []
initial_detection_semgrep_count: 0
initial_detection_semgrep_cwes: []
initial_detection_ast_count: 2
initial_detection_ast_cwes: ['022', '703']
iteration_detection_bandit_count: 0
iteration_detection_bandit_cwes: []
iteration_detection_semgrep_count: 0
iteration_detection_semgrep_cwes: []
iteration_detection_ast_count: 0
iteration_detection_ast_cwes: []

## Data Validation

Perform basic validation checks on the prepared dataset.

In [29]:
print("=" * 80)
print("DATA VALIDATION")
print("=" * 80)

# Check for duplicate workflow_ids
duplicates = df[df.duplicated(subset=['workflow_id'], keep=False)]
if len(duplicates) > 0:
    print(f"\n⚠️  Warning: Found {len(duplicates)} duplicate workflow_ids")
    print(duplicates[['workflow_id', 'timestamp', 'file']].head())
else:
    print("\n✓ No duplicate workflow_ids found")

# Check for missing critical fields
critical_fields = ['workflow_id', 'workflow', 'timestamp']
missing_critical = df[critical_fields].isnull().sum()
if missing_critical.sum() > 0:
    print(f"\n⚠️  Warning: Missing values in critical fields:")
    print(missing_critical[missing_critical > 0])
else:
    print("\n✓ All critical fields are populated")

# Check data consistency
print("\n" + "=" * 80)
print("CONSISTENCY CHECKS")
print("=" * 80)

# 1. Verify total vulnerabilities (fixed + remaining = identified)
print("\n1. Total Vulnerability Counts (may include duplicates):")
consistency_check = (
    df['total_vulnerabilities_fixed'] + df['total_vulnerabilities_remaining'] 
    == df['total_vulnerabilities_identified']
)
inconsistent = df[~consistency_check]
if len(inconsistent) > 0:
    print(f"  ⚠️  {len(inconsistent)} records have inconsistent total vulnerability counts")
    print("\n  Sample inconsistent records:")
    print(inconsistent[['workflow_id', 'total_vulnerabilities_identified', 
                        'total_vulnerabilities_fixed', 'total_vulnerabilities_remaining']].head())
else:
    print("  ✓ Total vulnerability counts are consistent")

# 2. Verify unique CWE counts (fixed + remaining = identified)
print("\n2. Unique CWE Counts:")
if 'unique_cwes_identified' in df.columns:
    unique_consistency_check = (
        df['unique_cwes_fixed'] + df['unique_cwes_remaining'] 
        == df['unique_cwes_identified']
    )
    inconsistent_unique = df[~unique_consistency_check]
    if len(inconsistent_unique) > 0:
        print(f"  ⚠️  {len(inconsistent_unique)} records have inconsistent unique CWE counts")
        print("\n  Sample inconsistent records:")
        print(inconsistent_unique[['workflow_id', 'unique_cwes_identified', 
                                    'unique_cwes_fixed', 'unique_cwes_remaining']].head())
    else:
        print("  ✓ Unique CWE counts are consistent (fixed + remaining = identified)")
else:
    print("  ⚠️  unique_cwes_identified column not found")

# 3. Verify fix provider counts match total fixed
print("\n3. Fix Provider Counts:")
fix_provider_totals = df['fix_provider_llm'] + df['fix_provider_rule_based'] + df['fix_provider_unknown']
fix_consistency_check = (fix_provider_totals == df['unique_cwes_fixed'])
inconsistent_fix = df[~fix_consistency_check]
if len(inconsistent_fix) > 0:
    print(f"  ⚠️  {len(inconsistent_fix)} records have mismatched fix provider counts")
    print("\n  Sample records with mismatch:")
    print(inconsistent_fix[['workflow_id', 'unique_cwes_fixed', 'fix_provider_llm', 
                            'fix_provider_rule_based', 'fix_provider_unknown']].head())
    print("\n  Detailed analysis of mismatches:")
    for idx, row in inconsistent_fix.head(3).iterrows():
        print(f"\n  Workflow: {row['workflow_id']}")
        print(f"    Unique CWEs fixed: {row['unique_cwes_fixed']}")
        print(f"    Fix provider sum: {row['fix_provider_llm'] + row['fix_provider_rule_based'] + row['fix_provider_unknown']}")
        print(f"    LLM fixes: {row['fix_provider_llm']}")
        print(f"    Rule-based fixes: {row['fix_provider_rule_based']}")
        print(f"    Unknown fixes: {row['fix_provider_unknown']}")
        if 'fixed_cwes' in df.columns:
            print(f"    Fixed CWEs: {row['fixed_cwes']}")
        if 'fix_provider_llm_cwes' in df.columns:
            print(f"    LLM CWEs: {row['fix_provider_llm_cwes']}")
        if 'fix_provider_rule_cwes' in df.columns:
            print(f"    Rule CWEs: {row['fix_provider_rule_cwes']}")
else:
    print("  ✓ Fix provider counts match unique CWEs fixed")

# Check for negative values
print("\n4. Negative Values Check:")
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
negative_values = (df[numeric_cols] < 0).sum()
if negative_values.sum() > 0:
    print(f"  ⚠️  Found negative values in:")
    print(negative_values[negative_values > 0])
else:
    print("  ✓ No negative values found")

print("\n" + "=" * 80)
print("Validation complete!")
print("=" * 80)

DATA VALIDATION

✓ No duplicate workflow_ids found

✓ All critical fields are populated

CONSISTENCY CHECKS

1. Total Vulnerability Counts (may include duplicates):
  ✓ Total vulnerability counts are consistent

2. Unique CWE Counts:
  ✓ Unique CWE counts are consistent (fixed + remaining = identified)

3. Fix Provider Counts:
  ⚠️  20 records have mismatched fix provider counts

  Sample records with mismatch:
        workflow_id  unique_cwes_fixed  fix_provider_llm  fix_provider_rule_based  fix_provider_unknown
2   20251219_154533                  1                 3                        0                     0
5   20251219_164453                  3                 3                        1                     0
18  20251219_170511                  2                 8                        0                     0
19  20251219_170947                  0                 0                       12                     0
22  20251220_184438                  2                 2         

## Understanding Duplicates: Total vs Unique CWE Counts

This section explains the difference between total vulnerability counts and unique CWE counts:

- **Total Vulnerabilities**: Count ALL occurrences (may include duplicate CWEs from different tools or iterations)
- **Unique CWEs**: Count each CWE ID only once per workflow

The fix provider counts (LLM, rule-based) are based on **unique CWEs**, so they should sum to `unique_cwes_fixed`, not `total_vulnerabilities_fixed`.

In [30]:
print("=" * 80)
print("TOTAL vs UNIQUE CWE COMPARISON")
print("=" * 80)

# Calculate statistics
print("\nDataset-wide Statistics:")
print(f"  Total records: {len(df)}")
print(f"\n  Total Vulnerability Counts (includes duplicates):")
print(f"    - Identified: {df['total_vulnerabilities_identified'].sum()}")
print(f"    - Fixed: {df['total_vulnerabilities_fixed'].sum()}")
print(f"    - Remaining: {df['total_vulnerabilities_remaining'].sum()}")

if 'unique_cwes_identified' in df.columns:
    print(f"\n  Unique CWE Counts (no duplicates):")
    print(f"    - Identified: {df['unique_cwes_identified'].sum()}")
    print(f"    - Fixed: {df['unique_cwes_fixed'].sum()}")
    print(f"    - Remaining: {df['unique_cwes_remaining'].sum()}")

print(f"\n  Fix Provider Counts (based on unique CWEs):")
print(f"    - LLM fixes: {df['fix_provider_llm'].sum()}")
print(f"    - Rule-based fixes: {df['fix_provider_rule_based'].sum()}")
print(f"    - Unknown fixes: {df['fix_provider_unknown'].sum()}")
print(f"    - Total: {df['fix_provider_llm'].sum() + df['fix_provider_rule_based'].sum() + df['fix_provider_unknown'].sum()}")

# Show records with duplicates (where total > unique)
if 'unique_cwes_identified' in df.columns:
    has_duplicates = df[df['total_vulnerabilities_identified'] > df['unique_cwes_identified']]
    print(f"\n  Records with duplicate CWE detections: {len(has_duplicates)}")
    
    if len(has_duplicates) > 0:
        print(f"\n  Average duplication factor: {has_duplicates['total_vulnerabilities_identified'].sum() / has_duplicates['unique_cwes_identified'].sum():.2f}x")
        print(f"\n  Sample records with duplicates:")
        sample = has_duplicates[['workflow_id', 'total_vulnerabilities_identified', 
                                 'unique_cwes_identified', 'total_vulnerabilities_fixed', 
                                 'unique_cwes_fixed']].head(5)
        print(sample.to_string(index=False))

print("\n" + "=" * 80)
print("Analysis complete!")
print("=" * 80)
print("\nRecommendation: Use 'unique_cwes_*' columns for accurate analysis.")
print("The fix provider counts are correctly aligned with unique CWE counts.")

TOTAL vs UNIQUE CWE COMPARISON

Dataset-wide Statistics:
  Total records: 84

  Total Vulnerability Counts (includes duplicates):
    - Identified: 215
    - Fixed: 202
    - Remaining: 13

  Unique CWE Counts (no duplicates):
    - Identified: 59
    - Fixed: 48
    - Remaining: 11

  Fix Provider Counts (based on unique CWEs):
    - LLM fixes: 47
    - Rule-based fixes: 112
    - Unknown fixes: 0
    - Total: 159

  Records with duplicate CWE detections: 27

  Average duplication factor: 4.39x

  Sample records with duplicates:
    workflow_id  total_vulnerabilities_identified  unique_cwes_identified  total_vulnerabilities_fixed  unique_cwes_fixed
20251219_154533                                 3                       1                            3                  1
20251219_164453                                 6                       3                            6                  3
20251219_165024                                 5                       3                         

## Summary of Changes

### Problem Addressed
Previously, there was a mismatch between:
- `total_vulnerabilities_fixed` (counted all vulnerability occurrences including duplicates)
- `fix_provider_llm` + `fix_provider_rule_based` (counted unique CWEs fixed)

### Solution Implemented

1. **New Function**: `count_fix_providers_by_cwe()`
   - Tracks which unique CWEs were fixed by each method (LLM, rule-based, unknown)
   - Returns counts and lists of unique CWEs per fix provider

2. **New Columns Added**:
   - `unique_cwes_identified`: Total unique CWEs found in workflow
   - `unique_cwes_fixed`: Total unique CWEs successfully fixed
   - `unique_cwes_remaining`: Total unique CWEs not fixed
   - `fix_provider_llm_cwes`: List of unique CWEs fixed by LLM
   - `fix_provider_rule_cwes`: List of unique CWEs fixed by rules
   - `fix_provider_unknown_cwes`: List of unique CWEs fixed by unknown method

3. **Enhanced Validation**:
   - Verifies `unique_cwes_fixed` + `unique_cwes_remaining` = `unique_cwes_identified`
   - Verifies `fix_provider_llm` + `fix_provider_rule_based` + `fix_provider_unknown` = `unique_cwes_fixed`
   - Provides detailed diagnostics when mismatches occur

### Usage Guidance

- **For counting vulnerabilities by occurrence**: Use `total_vulnerabilities_*` columns
- **For counting distinct CWE types**: Use `unique_cwes_*` columns
- **For analysis of fix effectiveness**: Use `unique_cwes_*` columns (aligns with fix provider counts)